In [54]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, YearLocator, DateFormatter
from matplotlib.ticker import MultipleLocator
from matplotlib.animation import FuncAnimation
import sqlite3
from datetime import datetime
import plotly.graph_objs as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from flask import Flask, g
from dash import dcc, html, Input, Output, Dash
from dash.exceptions import PreventUpdate


In [55]:
# Connect to the SQLite database
conn = sqlite3.connect('data/sp500_data.db')

In [56]:
# Get all the symbols in the StockData table
symbols_query = "SELECT DISTINCT Symbol FROM stocks"
symbols = [row[0] for row in conn.execute(symbols_query)]

In [57]:
# Initialize the Dash app
app = dash.Dash(__name__)

In [58]:
app.layout = html.Div([
    dcc.Input(id='stock-symbol', value='Enter Stock Symbol', type='text', debounce=True),
    dcc.Input(id='start-date', value='', placeholder='Enter Start Date (YYYY-MM-DD)', type='text', debounce=True),
    dcc.Input(id='end-date', value='', placeholder='Enter End Date (YYYY-MM-DD)', type='text', debounce=True),  
    dcc.Dropdown(
        id='time-frame',
        options=[
            {'label': '1 Month', 'value': '1M'},
            {'label': '6 Months', 'value': '6M'},
            {'label': '1 Year', 'value': '1Y'},
            {'label': '3 Years', 'value': '3Y'},
            {'label': '5 Years', 'value': '5Y'},
        ],
        value='1M',
    ),
    dcc.Graph(id='plot'),
])


In [59]:
start_date = "2017-01-01"
end_date = "2023-01-01"
stock_symbol = "MMM"

In [60]:
@app.callback(
    Output('plot', 'figure'),
    [
        Input('stock-symbol', 'value'),
        Input('start-date', 'value'),
        Input('end-date', 'value'),
        Input('time-frame', 'value')
    ]
)
def update_graph(stock_symbol, start_date_str, end_date_str, time_frame):
    # Convert the input strings to datetime objects
    start_date = pd.to_datetime(start_date_str)
    end_date = pd.to_datetime(end_date_str)

    # Construct the SQL query for the StockData table
    sql_query_stocks = f"SELECT * FROM stocks WHERE Symbol = ? AND Date >= ? AND Date <= ?"

    # Execute the query for the StockData table with user input
    cursor_stocks = conn.cursor()
    cursor_stocks.execute(sql_query_stocks, (stock_symbol, start_date_str, end_date_str))

    # Fetch all the rows from the StockData table
    rows_stocks = cursor_stocks.fetchall()

    # Create a DataFrame from the query results of the StockData table
    columns_stocks = [description[0] for description in cursor_stocks.description]
    df_stockdata = pd.DataFrame(rows_stocks, columns=columns_stocks)

    # Add a percent change column to the DataFrame for the StockData table
    df_stockdata['Percent_Change'] = df_stockdata['Adj_Close'].pct_change() * 100

    # Drop any NaN values in the Percent_Change column for the StockData table
    df_stockdata.dropna(subset=['Percent_Change'], inplace=True)

    # Calculate the cumulative percent change for the stock symbol
    df_stockdata['Cumulative_Percent_Change'] = ((1 + df_stockdata['Adj Close'].pct_change().fillna(0)).cumprod() * 100)-100
    # Construct the SQL query for the SP500 table
    sql_query_sp500 = f"SELECT * FROM sp500 WHERE Date >= ? AND Date <= ?"

    # Execute the query for the SP500 table with user input
    cursor_sp500 = conn.cursor()
    cursor_sp500.execute(sql_query_sp500, (start_date_str, end_date_str))

    # Fetch all the rows from the SP500 table
    rows_sp500 = cursor_sp500.fetchall()

    # Create a DataFrame from the query results of the SP500 table
    columns_sp500 = [description[0] for description in cursor_sp500.description]
    df_sp500 = pd.DataFrame(rows_sp500, columns=columns_sp500)

    # Add a percent change column to the DataFrame for the SP500 table
    df_sp500['Percent_Change'] = df_sp500['SP500_Price'].pct_change() * 100

    # Drop any NaN values in the Percent_Change column for the SP500 table
    df_sp500.dropna(subset=['Percent_Change'], inplace=True)

    # Calculate the cumulative percent change for the SP500
    df_sp500['Cumulative_Percent_Change'] = ((1 + df_sp500['SP500_Price'].pct_change().fillna(0)).cumprod() * 100)-100
    # Create a Plotly figure
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces for the stock and SP500
    fig.add_trace(
        go.Scatter(x=df_stockdata['Date'], y=df_stockdata['Cumulative_Percent_Change'], name=f"{stock_symbol} Cumulative Percent Change", mode='lines'),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df_sp500['Date'], y=df_sp500['Cumulative_Percent_Change'], name="S&P 500 Cumulative Percent Change", mode='lines'),
        secondary_y=True,
    )

    # Add title and labels
    fig.update_layout(
        title=f'{stock_symbol} Cumulative Percent Change compared to S&P 500 from {start_date_str} to {end_date_str}',
        xaxis_title='Date',
        yaxis_title=f'{stock_symbol} Cumulative Percent Change',
        yaxis2_title='S&P 500 Cumulative Percent Change',
    )
    conn.close()
    return fig  


In [61]:
if __name__ == '__main__':
    app.run_server(debug=True, threaded=False)

---------------------------------------------------------------------------
ProgrammingError                          Traceback (most recent call last)
Cell In[60], line 19, in update_graph(
    stock_symbol='Enter Stock Symbol',
    start_date_str='',
    end_date_str='',
    time_frame='1M'
)
     16 sql_query_stocks = f"SELECT * FROM stocks WHERE Symbol = ? AND Date >= ? AND Date <= ?"
     18 # Execute the query for the StockData table with user input
---> 19 cursor_stocks = conn.cursor()
        conn = <sqlite3.Connection object at 0x000001A68ACB7140>
     20 cursor_stocks.execute(sql_query_stocks, (stock_symbol, start_date_str, end_date_str))
     22 # Fetch all the rows from the StockData table

ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 30500 and this is thread id 13640.

---------------------------------------------------------------------------
ProgrammingError                          Traceba